In [13]:
from bs4 import BeautifulSoup
import pandas as pd 
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

In [23]:
url_unique ='https://www.kammi.it'
url = "https://www.kammi.it/scarpe-donna/stivali-stivaletti/"
header = {"User-Agent":"Chrome/39.0"}
print(url)
page = requests.get(url, headers=header)
print(page)

https://www.kammi.it/scarpe-donna/stivali-stivaletti/
<Response [200]>


In [24]:
soup = BeautifulSoup(page.text)
stivnomi = soup.find_all('div', class_="container-fluid product-table")
prod_stivali = []

In [29]:
# Configura il browser in modalità headless
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Esegui in modalità headless (senza UI)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Lista per salvare i dati estratti da ciascun prodotto
prodotti_dati = []

# Itera su tutti i link nella lista `stivnomi`
for stivale in stivnomi:
    try:
        # Ottieni il link del prodotto
        url_stivale = stivale.get('href')  # Supponendo che `stivnomi` contenga elementi HTML
        url1 = url_unique + url_stivale    # Costruisci l'URL completo
        
        # Carica la pagina
        driver.get(url1)

        # Aspetta che JavaScript carichi la pagina (puoi aggiustare il tempo)
        time.sleep(5)

        # Estrai il contenuto della pagina
        html_content = driver.page_source

        # Analizza la pagina con BeautifulSoup
        soup_stivale = BeautifulSoup(html_content, 'html.parser')

        # Estrai i dati necessari
        name_stiv = soup_stivale.find('h1', class_='"text-black h4 fw-bold"')
        colore_stiv = soup_stivale.find('span', class_="colore")
        prezzo_scontato_stiv = soup_stivale.find('p', class_="price mt-2")
        prezzo_originale_stiv = soup_stivale.find('p', class_="old-price d-inline-block mr-3")
        colors_divs = soup_stivale.find_all('div', class_='related__color-name skeleton-effect')
        sizes_list = soup_stivale.find_all('div', class_="product-options-wrap d-flex flex-wrap align-items-center")

        # Gestione valori (evita errori in caso di dati mancanti)
        nomi = name_stiv.text.strip() if name_stiv else "N/A"
        colore = colore_stiv.text.strip() if colore_stiv else "N/A"
        prezzo_scontato = prezzo_scontato_stiv.text.replace("€","").strip() if prezzo_scontato_stiv else "N/A"
        prezzo_originale = prezzo_originale_stiv.text.replace("€","").strip() if prezzo_originale_stiv else "N/A"
        
        # Rimuovi il colore dal nome per ottenere solo il nome del prodotto
        if colore in nomi:
            nomi = nomi.replace(colore, "").strip()

        # Estrai tutti i colori disponibili
        all_colors = [color_div.text.strip() for color_div in colors_divs if color_div.text.strip()]
        
        #  Estrai tutte le taglie disponibili
        sizes_list = soup_stivale.find_all('ul', class_='related__select-optgroup-options')  # Aggiorna la classe se necessario

        # Debug: verifica se sizes_list contiene elementi
        if not sizes_list:
            print(f"Taglie non trovate per il prodotto {url1}")
        
        # Estrai il testo da ogni elemento e filtra solo i numeri
        import re  # Libreria per espressioni regolari
        all_sizes = []
        for ul in sizes_list:
            sizes = ul.find_all('li')  # Supponendo che ogni taglia sia un elemento <li>
            for size in sizes:
                size_text = size.text.strip()
                # Filtra solo numeri usando una regex
                numbers = re.findall(r'\b\d+\b', size_text)  # Cerca numeri interi (come 35, 36)
                all_sizes.extend(numbers)
        
        # Rimuovi duplicati e ordina (opzionale)
        all_sizes = sorted(set(all_sizes), key=int)
        
        # Unisci le taglie in una stringa separata da virgole
        taglie_disponibili = ", ".join(all_sizes) if all_sizes else "N/A"
        
        # Salva i dati per questo prodotto
        prodotti_dati.append({
            'nome': nomi,
            'colore': colore,
            'prezzo_scontato': prezzo_scontato,
            'prezzo_originale': prezzo_originale,
            'colori_disponibili': ", ".join(all_colors),
            'taglie_disponibili': taglie_disponibili,
            'categoria' : "stivaletti",
            'gender':'F',
            'url': url1
        })

    except Exception as e:
        print(f"Errore nell'elaborare il prodotto {url1}: {e}")

# Chiudi il browser
driver.quit()

# Crea un DataFrame con i dati estratti
df_prodotti = pd.DataFrame(prodotti_dati)


# Visualizza il DataFrame
df_prodotti


Errore nell'elaborare il prodotto None: Message: invalid argument: 'url' must be a string
  (Session info: chrome=131.0.6778.86)
Stacktrace:
	GetHandleVerifier [0x003C3433+25059]
	(No symbol) [0x0034CE34]
	(No symbol) [0x0022BEC3]
	(No symbol) [0x002AC006]
	(No symbol) [0x00291EFC]
	(No symbol) [0x002AB51E]
	(No symbol) [0x00291C96]
	(No symbol) [0x00263FAC]
	(No symbol) [0x00264F3D]
	GetHandleVerifier [0x006B5593+3113795]
	GetHandleVerifier [0x006CA25A+3198986]
	GetHandleVerifier [0x006C2A32+3168226]
	GetHandleVerifier [0x004632A0+680016]
	(No symbol) [0x0035577D]
	(No symbol) [0x00352A28]
	(No symbol) [0x00352BC5]
	(No symbol) [0x00345820]
	BaseThreadInitThunk [0x76CD7BA9+25]
	RtlInitializeExceptionChain [0x771EC0CB+107]
	RtlClearBits [0x771EC04F+191]



""


In [35]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import re  # Libreria per gestire espressioni regolari

# Configura le URL e l'header
url_unique = 'https://www.kammi.it'
url = "https://www.kammi.it/scarpe-donna/stivali-stivaletti/"
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"}

# Effettua una richiesta alla pagina
page = requests.get(url, headers=headers)
page.raise_for_status()  # Verifica che la richiesta sia andata a buon fine
soup = BeautifulSoup(page.text, 'html.parser')

# Trova tutti i contenitori dei prodotti
stivnomi = soup.find_all('div', class_="product-box")  # Aggiorna il selettore secondo la struttura HTML

# Configura il browser in modalità headless
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Esegui in modalità headless
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Lista per salvare i dati estratti da ciascun prodotto
prodotti_dati = []

# Itera su tutti i contenitori di prodotto trovati
for prodotto in stivnomi:
    try:
        # Ottieni il link del prodotto
        link_tag = prodotto.find('a', href=True)
        if not link_tag:
            continue
        url_stivale = link_tag['href']
        url1 = url_stivale if url_stivale.startswith('http') else url_unique + url_stivale  # Gestisci link relativi
        
        # Carica la pagina del prodotto con Selenium
        driver.get(url1)
        time.sleep(5)  # Attendi che la pagina venga caricata completamente
        html_content = driver.page_source
        soup_stivale = BeautifulSoup(html_content, 'html.parser')

        # Estrai i dati necessari
        name_stiv = soup_stivale.find('p', class_='product-title')
        colore_stiv = soup_stivale.find('span', class_='colore')
        prezzo_scontato_stiv = soup_stivale.find('p', class_='price mt-2')
        prezzo_originale_stiv = soup_stivale.find('p', class_='old-price d-inline-block mr-3')
        colors_divs = soup_stivale.find_all('div', class_='related__color-name skeleton-effect')
        sizes_list = soup_stivale.find_all('div', class_="product-options-wrap d-flex flex-wrap align-items-center")

        # Pulizia e gestione dei dati mancanti
        nomi = name_stiv.text.strip() if name_stiv else "N/A"
        colore = colore_stiv.text.strip() if colore_stiv else "N/A"
        prezzo_scontato = prezzo_scontato_stiv.text.replace("€", "").strip() if prezzo_scontato_stiv else "N/A"
        prezzo_originale = prezzo_originale_stiv.text.replace("€", "").strip() if prezzo_originale_stiv else "N/A"

        # Estrai tutti i colori disponibili
        all_colors = [color_div.text.strip() for color_div in colors_divs if color_div.text.strip()]

        # Estrai tutte le taglie disponibili e filtra solo i numeri
        all_sizes = []
        for ul in sizes_list:
            sizes = ul.find_all('li')
            for size in sizes:
                size_text = size.text.strip()
                numbers = re.findall(r'\b\d+\b', size_text)  # Cerca solo numeri
                all_sizes.extend(numbers)

        # Rimuovi duplicati e ordina le taglie
        all_sizes = sorted(set(all_sizes), key=int)
        taglie_disponibili = ", ".join(all_sizes) if all_sizes else "N/A"

        # Salva i dati per questo prodotto
        prodotti_dati.append({
            'nome': nomi,
            'colore': colore,
            'prezzo_scontato': prezzo_scontato,
            'prezzo_originale': prezzo_originale,
            'colori_disponibili': ", ".join(all_colors),
            'taglie_disponibili': taglie_disponibili,
            'categoria': "stivaletti",
            'gender': 'F',
            'url': url1
        })

    except Exception as e:
        print(f"Errore nell'elaborare il prodotto {url1}: {e}")

# Chiudi il browser
driver.quit()

# Crea un DataFrame con i dati estratti
df_prodotti = pd.DataFrame(prodotti_dati)

# Visualizza il DataFrame
print(df_prodotti)

#Salva i dati in un file CSV
df_prodotti.to_csv('kammi_stivali_st.csv', index=False)


              nome colore prezzo_scontato prezzo_originale colori_disponibili  \
0  Tronchetto 6022                 125,10           139,00                      
1  Tronchetto 6022                 125,10           139,00                      

  taglie_disponibili   categoria gender  \
0                N/A  stivaletti      F   
1                N/A  stivaletti      F   

                                                 url  
0  https://www.kammi.it/scarpe-donna/stivali-stiv...  
1  https://www.kammi.it/scarpe-donna/stivali-stiv...  


,nome,colore,prezzo_scontato,prezzo_originale,colori_disponibili,taglie_disponibili,categoria,gender,url
0,Tronchetto 6022,,"125,10","139,00",,N/A,stivaletti,F,https://www.kammi.it/scarpe-donna/stivali-stiv...
1,Tronchetto 6022,,"125,10","139,00",,N/A,stivaletti,F,https://www.kammi.it/scarpe-donna/stivali-stiv...
